In [1]:
# In this notebook, I will be implementing a Decision Tree Classifier from scratch, and training & evaluating it on the 
# Iris Dataset.

In [2]:
import numpy as np
import sklearn
from sklearn.datasets import load_iris

In [3]:
iris_data = load_iris()
print(iris_data.keys())
print(iris_data.feature_names)
print(iris_data.target_names)

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
['setosa' 'versicolor' 'virginica']


In [4]:
X = iris_data['data']
Y = iris_data['target']

print(X.shape, Y.shape)

(150, 4) (150,)


In [5]:
shuffled_indices = np.random.permutation(len(X))
train_set_size = int(0.8*len(X))

X_train = X[shuffled_indices[:train_set_size]]
X_test = X[shuffled_indices[train_set_size:]]
Y_train = Y[shuffled_indices[:train_set_size]]
Y_test = Y[shuffled_indices[train_set_size:]]

print(X_train.shape, Y_train.shape, '\n', X_test.shape, Y_test.shape)

(120, 4) (120,) 
 (30, 4) (30,)


In [6]:
def Classes(labels):
    d = {}
    for label in labels:
        if label not in d:
            d[label] = 0
        d[label] += 1
    m = len(labels)    
    for key in d.keys():
        d[key] = d[key]/m
    return d

def Value(labels):
    values = [0]*3
    for label in labels:
        values[int(label)] += 1
    return values

def GiniImpurity(labels):
    G = 1
    d = Classes(labels)
    for key in d.keys():
        G = G - d[key]**2
    
    return G

def SplitData(X, Y, feature, threshold):
    m = len(X)
    X_left = np.zeros((1,X.shape[1]))
    X_right = np.zeros((1,X.shape[1]))
    Y_left = np.zeros((1,1))
    Y_right = np.zeros((1,1))
    
    for i in range(m):
        if X[i,feature] <= threshold:
            X_left = np.vstack((X_left,X[i]))
            Y_left = np.vstack((Y_left,Y[i]))
        else:
            X_right = np.vstack((X_right,X[i]))
            Y_right = np.vstack((Y_right,Y[i]))
    X_left = X_left[1:,:]
    X_right = X_right[1:,:]
    Y_left = Y_left[1:,:]
    Y_right = Y_right[1:,:]
    return X_left,X_right,Y_left,Y_right

def CARTCostFunction(X, Y, feature, threshold):
    m = len(X)
    Y_left = np.zeros((1,1))
    Y_right = np.zeros((1,1))
    
    for i in range(m):
        if X[i,feature] <= threshold:
            Y_left = np.vstack((Y_left,Y[i]))
        else:
            Y_right = np.vstack((Y_right,Y[i]))
    Y_left = Y_left[1:,:]
    Y_right = Y_right[1:,:]
    
    m_left = len(Y_left)
    m_right = len(Y_right)
    
    G_left = GiniImpurity(Y_left.flatten().tolist())
    G_right = GiniImpurity(Y_right.flatten().tolist())
    
    cost = (m_left/m)*G_left + (m_right/m)*G_right
    return cost
    
    
print('Testing the "Classes" Function: ', Classes(np.array([0,0,0,1,1,1,1,2,2,2])))
print('Testing the "Value" Function: ', Value(np.array([0,0,0,1,1,1,1,2,2,2])))
print('Testing the "GiniImpurity" Function: ', GiniImpurity(np.array([0,0,0,1,1,1,1,2,2,2])))
print('Testing the "CARTCostFunction" Function: ', CARTCostFunction(X_train, Y_train, 2, 2.45))


Testing the "Classes" Function:  {0: 0.3, 1: 0.4, 2: 0.3}
Testing the "Value" Function:  [3, 4, 3]
Testing the "GiniImpurity" Function:  0.66
Testing the "CARTCostFunction" Function:  0.3416666666666667


In [7]:
class Node():
    def __init__(self, X, Y, parent = None, branch = "NaN"):
        self.X = X
        self.Y = Y
        self.parent = parent
        self.branch = branch
        
        self.child = []
        self.gini = float('nan')
        self.samples = float('nan')
        self.value = []
        self.feature = float('nan')
        self.threshold = float('nan')
        self.leaf = 'N'
        self.target = float('nan')

def PrintNodeAttributes(node):
    print('Gini: ', node.gini)
    print('Samples: ', node.samples)
    print('Value: ', node.value)
    print('Feature: ', node.feature)
    print('Threshold: ', node.threshold)
    print('Leaf: ', node.leaf)
    print('Class: ', node.target)

In [8]:
def CreateTree(X,Y):
    pure = []
    impure = []
    nodes = []

    node = Node(X,Y)
    node.gini = GiniImpurity(node.Y.flatten().tolist())
    node.samples = len(node.Y)
    node.value = Value(node.Y.flatten().tolist())

    if node.gini == 1:
        pure.append(node)
        node.leaf = "Y"
    else:
        impure.append(node)

    #print('List of nodes: ', nodes)
    #print('Leaf nodes: ', pure)
    #print('Impure nodes: ', impure)

    while impure != []:
        node = impure[0]
        if node.gini == 0:
            impure.pop(0)
            pure.append(node)
            nodes.append(node)
            node.leaf = "Y"
            node.target = node.value.index(max(node.value))
            #print('Pure Node')
        else:

            X = node.X
            Y = node.Y
            # Find the best split (feature, threshold)
            best_feature = float('nan')
            best_threshold = float('nan')
            min_cost = float('inf')

            features = np.arange(X.shape[1]).tolist()
            for feature in features:
                if len(X) == 1 or X[:,feature].min() == X[:,feature].max():
                    thresholds = list(X[:,feature])
                else:
                    thresholds = np.arange(X[:,feature].min(), X[:,feature].max(), (X[:,feature].max() - X[:,feature].min())/25).tolist() 
                for threshold in thresholds:
                    cost = CARTCostFunction(X,Y,feature,threshold)
                    if cost < min_cost:
                        best_feature = feature
                        best_threshold = threshold
                        min_cost = cost

            node.feature = best_feature
            node.threshold = best_threshold

            #print('List of nodes: ', nodes)
            #print('Leaf nodes: ', pure)
            #print('Impure nodes: ', impure)
            #print('\n')
            #PrintNodeAttributes(node)
            #print('\n')

            X_left,X_right,Y_left,Y_right = SplitData(node.X,node.Y,best_feature,best_threshold)

            impure.pop(0)

            nodeLeft = Node(X_left,Y_left,parent=node,branch='L')
            nodeRight = Node(X_right,Y_right,parent=node,branch='R')

            for temp_node in [nodeLeft,nodeRight]:
                node.child.append(temp_node)
                temp_node.gini = GiniImpurity(temp_node.Y.flatten().tolist())
                temp_node.samples = len(temp_node.Y)
                temp_node.value = Value(temp_node.Y.flatten().tolist())

                impure.append(temp_node)


            nodes.append(node)


            #print('List of nodes: ', nodes)
            #print('Leaf nodes: ', pure)
            #print('Impure nodes: ', impure)
            #print('\n')
    
    return nodes,pure,impure

nodes, pure, impure = CreateTree(X_train,Y_train)

In [9]:
for node in nodes:
    PrintNodeAttributes(node)
    print(node.parent)
    print(node.child)
    print('\n')

Gini:  0.6662499999999999
Samples:  120
Value:  [38, 41, 41]
Feature:  2
Threshold:  2.028
Leaf:  N
Class:  nan
None
[<__main__.Node object at 0x0000018A1A56EE48>, <__main__.Node object at 0x0000018A1A56EF88>]


Gini:  0.0
Samples:  38
Value:  [38, 0, 0]
Feature:  nan
Threshold:  nan
Leaf:  Y
Class:  0
[]


Gini:  0.5
Samples:  82
Value:  [0, 41, 41]
Feature:  2
Threshold:  4.740000000000001
Leaf:  N
Class:  nan
[<__main__.Node object at 0x0000018A1A532B48>, <__main__.Node object at 0x0000018A1A5320C8>]


Gini:  0.0
Samples:  36
Value:  [0, 36, 0]
Feature:  nan
Threshold:  nan
Leaf:  Y
Class:  1
[]


Gini:  0.19376181474480153
Samples:  46
Value:  [0, 5, 41]
Feature:  3
Threshold:  1.62
Leaf:  N
Class:  nan
[<__main__.Node object at 0x0000018A1A56B508>, <__main__.Node object at 0x0000018A1A56B108>]


Gini:  0.5
Samples:  8
Value:  [0, 4, 4]
Feature:  2
Threshold:  4.92
Leaf:  N
Class:  nan
[<__main__.Node object at 0x0000018A1A56B208>, <__main__.Node object at 0x0000018A1A56B2C8>]


Gi

In [10]:
def Predict(X_test, nodes):
    predictions = np.zeros(len(X_test))
    idx = 0
    for instance in X_test:
        node = nodes[0]
        while True:
            if node.leaf == 'Y':
                predictions[idx] = node.target
                break
            else:
                feature = node.feature
                threshold = node.threshold

                if instance[feature] <= threshold:
                    node = node.child[0]
                else:
                    node = node.child[1]
        idx += 1
    return predictions

Y_test_predictions = Predict(X_test, nodes)

In [11]:
def ModelPerformanceMetrics(Y,Y_predicted):
    correct = 0
    total = 0
    cm = np.zeros((3,3))
         
    for y,yp in zip(Y,Y_predicted):
        if y == yp:
            correct += 1
            cm[y,y] += 1
        else:
            cm[y,int(yp)] += 1            
        total += 1    
    
    
    accuracy = correct/total
    return accuracy, cm

accuracy, cm = ModelPerformanceMetrics(Y_test,Y_test_predictions)
print('Decision Tree Classifier Performance'.upper(), '\n')
print('Accuracy: ', '{} %'.format(accuracy*100))
print('Confusion Matrix: ', '\n', cm)

DECISION TREE CLASSIFIER PERFORMANCE 

Accuracy:  93.33333333333333 %
Confusion Matrix:  
 [[12.  0.  0.]
 [ 0.  8.  1.]
 [ 0.  1.  8.]]


In [12]:
'''This implementation performs about as well as the Scikit-Learn implementation'''

'This implementation performs about as well as the Scikit-Learn implementation'